In [30]:
import networkx as nx

In [31]:
def read_graph(benchmark):
  G = nx.DiGraph()

  node_data = open(f"../benchmarks/{benchmark}/nodes.txt", 'r').readlines()
  node_arr = []
  nd_nodes = []
  for line in node_data: 
    idx, commit_id, node_str, nondeterministic = line.split(',')
    node_arr.append((int(idx), {
        "commit_id" : int(commit_id), 
        "node_str" : node_str, 
        "nondeterministic" : bool(int(nondeterministic))
      }))
    if (int(nondeterministic)): nd_nodes.append(int(idx))
  G.add_nodes_from(node_arr)

  edge_data = open(f"../benchmarks/{benchmark}/edges.txt", 'r').readlines()
  for line in edge_data:
    src, dst = [int(i) for i in line.split(',')]
    G.add_edge(src, dst)

  return G, nd_nodes

In [63]:
def bfs_nodes(G, source): 
  bfs_out = list(nx.dfs_edges(G, source))
  return set([x[1] for x in bfs_out])

def multi_bfs(G, sources): 
  # G_cp = nx.DiGraph(G)
  bfs_out = set()
  for source in sources: 
    if source in bfs_out: continue
    src_out = bfs_nodes(G, source)
    bfs_out = bfs_out.union(src_out)
    # G_cp.remove_nodes_from(src_out)
  
  return bfs_out

In [75]:
from collections import deque

def generic_bfs_edges(G, sources, neighbors=None, depth_limit=None, sort_neighbors=None):
    if callable(sort_neighbors):
        _neighbors = neighbors
        neighbors = lambda node: iter(sort_neighbors(_neighbors(node)))

    visited = set(sources)
    if depth_limit is None:
        depth_limit = len(G)
    queue = deque([(source, depth_limit, neighbors(source)) for source in sources])
    while queue:
        parent, depth_now, children = queue[0]
        try:
            child = next(children)
            if child not in visited:
                yield parent, child
                visited.add(child)
                if depth_now > 1:
                    queue.append((child, depth_now - 1, neighbors(child)))
        except StopIteration:
            queue.popleft()

def bfs_edges(G, sources, reverse=False, depth_limit=None, sort_neighbors=None):
    if reverse and G.is_directed():
        successors = G.predecessors
    else:
        successors = G.neighbors
    yield from generic_bfs_edges(G, sources, successors, depth_limit, sort_neighbors)

def bfs_multi_src(G, sources):
    return [x[1] for x in list(bfs_edges(G, sources))]

In [79]:
G, nd_nodes = read_graph("vsra")
# print(multi_bfs(G, nd_nodes))

In [84]:
x = multi_bfs(G, nd_nodes)

In [85]:
y = bfs_multi_src(G, nd_nodes)

In [89]:
for item in x: 
  if item not in y: print(item)
print()
for item in y:
  if item not in x: print(item)